In [2]:

from IPython.display import HTML, display
import ipywidgets as widgets
import re
import subprocess
import os

# Create a text input widget for the video URL
url_input = widgets.Text(
    value='',
    placeholder='Enter the video URL here',
    description='Video URL:',
    layout=widgets.Layout(width='100%')
)

# Create a button to process the URL
process_button = widgets.Button(description="Process URL")
output_area = widgets.Output()

# Display the widgets
display(url_input)
display(process_button)
display(output_area)

# Function to process the video URL
def process_video_url(b):
    with output_area:
        output_area.clear_output()
        video_url = url_input.value.strip()
        if not video_url:
            print("Please enter a valid video URL.")
            return

        try:
            # Download the subtitles using ffmpeg
            subprocess.run(['ffmpeg', '-i', video_url, '-y', '/content/sub.srt'], check=True)

            # Check if the subtitle file exists
            if os.path.exists('/content/sub.srt'):
                # Read and process the SRT file
                with open('/content/sub.srt', 'r') as input_file:
                    srt = input_file.read()

                # Remove tags and unwanted content
                srt = re.sub(r'<.+?>', '', srt)
                srt = re.sub(r'{.+?}', '', srt)
                srt = re.sub(r'^\d+\n\d+:\d+:\d+,\d+ --> \d+:\d+:\d+,\d+\n', '', srt, flags=re.M)
                srt = re.sub('\n', ' ', srt)
                srt = re.sub(r' ‏', ' ', srt)
                srt = re.sub(r'  ', ' ', srt)
                srt = re.sub(r'\. ', '.\n\n', srt)
                srt = re.sub(r'\? ', '?\n\n', srt)
                srt = re.sub(r'\.” ', '.”\n\n', srt)
                srt = re.sub('  ', ' ', srt)
                srt = re.sub(r'\.\n\n\.\n\n\.\n\n', '. . . ', srt)
                srt = re.sub(r'\.\n\n\.\n\n\.”', '. . .”', srt)
                srt = re.sub(r'\.\n\.\n\.\n', '. . . ', srt)

                # Create the HTML structure
                html_content = f"""
                <div style="white-space: pre-wrap; word-wrap: break-word; font-family: Arial, sans-serif; line-height: 1.5; padding: 10px;">
                {srt}
                </div>
                <button id="copy-button" style="background-color:#4CAF50; color:white; padding:5px 10px; border:none; cursor:pointer;">
                Copy Text
                </button>
                <textarea id="copy-text" style="display:none;">{srt}</textarea>
                <script>
                document.querySelector("#copy-button").addEventListener("click", function() {{
                    var copyText = document.querySelector("#copy-text");
                    copyText.style.display = "block";
                    copyText.select();
                    document.execCommand("copy");
                    copyText.style.display = "none";
                    alert("Text copied to clipboard!");
                }});
                </script>
                """
                display(HTML(html_content))

                # Clean up
                os.remove('/content/sub.srt')
            else:
                print("The video does not have an associated subtitle file.")
        except Exception as e:
            print(f"An error occurred: {e}")

# Link the button to the processing function
process_button.on_click(process_video_url)

Text(value='', description='Video URL:', layout=Layout(width='100%'), placeholder='Enter the video URL here')

Button(description='Process URL', style=ButtonStyle())

Output()